In [13]:
import os
import sys
import pandas as pd
import nltk, re, string, collections

from nltk import ngrams
from pandas import read_csv
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from nltk.stem import WordNetLemmatizer 


pd.set_option("display.max_columns", 100)

# Se pueden usar 'english' o 'spanish'
stop_words = set(stopwords.words('spanish'))  
lemmatizer = WordNetLemmatizer() 


In [100]:
fileName = 'facts_prediction_dataset_1617483308628'

# Load dataset inicial
# El objetivo es procesar primero los campos de commits, adrs y chat/emails y dejar solo las palabras mas relevantes
df_ = read_csv('/Users/borisperezg/rebelmodels_storing/datasets_to_classify/'+fileName+'.csv', delimiter="|")

In [101]:
df_

,factid,drivertype,goaltype,sourceelementname,sourceelementtype,layersource,targetelementname,targetelementtype,layertarget,isnewelement,iscyclic,facttype,relatontype,actiontype,incoming,outcoming,ratiolinks,mostlinkedlayer,commitlogs,adrlogs,chatlogs,property,propertynewvalue,propertyoldvalue
0,127,improve process,minimal process definition,production service,service,business,NaN,NaN,NaN,NaN,False,element,NaN,create,1,1,0.17,business,NaN,NaN,NaN,NaN,NaN,NaN
1,128,improve process,minimal process definition,production process,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,0,4,0.33,business,NaN,NaN,NaN,NaN,NaN,NaN
2,129,improve process,minimal process definition,production planning,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,1,1,0.17,business,NaN,NaN,NaN,NaN,NaN,NaN
3,130,improve process,minimal process definition,processing,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,2,1,0.25,business,NaN,NaN,NaN,NaN,NaN,NaN
4,131,improve process,minimal process definition,packaging,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,2,1,0.25,business,NaN,NaN,NaN,NaN,NaN,NaN
5,132,improve process,minimal process definition,delivering process,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,1,0,0.08,business,NaN,NaN,NaN,NaN,NaN,NaN
6,133,improve process,minimal process definition,production service,service,business,simplification of product manufacturing process,requirement,motivation,True,False,relation,realization,create,0,0,0.00,business,NaN,NaN,NaN,NaN,NaN,NaN
7,134,improve process,minimal process definition,production process,process,business,production planning,process,business,True,False,relation,composition,create,0,0,0.00,business,NaN,NaN,NaN,NaN,NaN,NaN
8,135,improve process,minimal process definition,production process,process,business,processing,process,business,True,False,relation,composition,create,0,0,0.00,business,NaN,NaN,NaN,NaN,NaN,NaN
9,136,improve process,minimal process definition,production process,process,business,packaging,process,business,True,False,relation,composition,create,0,0,0.00,business,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# ---------------------------------------------------------------------
# DEFINICION DE FUNCIONES DE REMOCION DE STOP WORDS Y LEMATIZACION
# ---------------------------------------------------------------------

In [103]:
def remove_stopwords(newText):
    word_tokens = word_tokenize(newText.lower())  
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
  
    filtered_sentence = []  
  
    for w in word_tokens:  
        # Excluyendo los stopwords
        if w not in stop_words:  
            # Excluyendo puntuaciones
            if w.isalpha():
                filtered_sentence.append(lemmatizer.lemmatize(w))
            
    return " ".join(filtered_sentence)

def bestNGrams(newText):

    my_ngrams = ngrams(newText.split(), 2)
    esBigramFreq = collections.Counter(my_ngrams)
    
    xyz = ''
    
    for letter, count in esBigramFreq.most_common(5):
        xyz += ' '.join(letter)
        xyz += ' '
    
    return xyz

In [104]:
# ------------------------------------------------------------
# PROCESAMIENTO DE CAMPOS COMMITLOGS, ADR Y CHATLOGS
# ------------------------------------------------------------

In [105]:
df_[['commitlogs']] = df_[['commitlogs']].fillna(value='NoData')
df_['commitlogs_lemma'] = df_.commitlogs.apply(remove_stopwords)
df_['commitlogs_ngrams'] = df_.commitlogs_lemma.apply(bestNGrams)

In [106]:
df_[['chatlogs']] = df_[['chatlogs']].fillna(value='NoData')
df_['chatlogs_lemma'] = df_.chatlogs.apply(remove_stopwords)
df_['chatlogs_ngrams'] = df_.chatlogs_lemma.apply(bestNGrams)

In [107]:
df_[['adrlogs']] = df_[['adrlogs']].fillna(value='NoData')
df_['adrlogs_lemma'] = df_.adrlogs.apply(remove_stopwords)
df_['adrlogs_ngrams'] = df_.adrlogs_lemma.apply(bestNGrams)

In [108]:
# ------------------------------------------------------------
# BORRANDO COLUMNAS YA PROCESADAS
# ------------------------------------------------------------

In [109]:
df_.drop(['commitlogs'], axis=1, inplace = True)
df_.drop(['commitlogs_lemma'], axis=1, inplace = True)

df_.drop(['chatlogs'], axis=1, inplace = True)
df_.drop(['chatlogs_lemma'], axis=1, inplace = True)

df_.drop(['adrlogs'], axis=1, inplace = True)
df_.drop(['adrlogs_lemma'], axis=1, inplace = True)

In [110]:
# ------------------------------------------------------------------------------
# SE ALMACENA EL ARCHIVO LISTO PARA SER USADO PARA ENTRENAR O IDENTIFICAR
# ------------------------------------------------------------------------------

In [111]:
df_.to_csv('/Users/borisperezg/rebelmodels_storing/datasets_to_classify/'+fileName+'_pa.csv', index=False, index_label=True)